In [1]:
#seq2seq:기계번역
#영어단어 -> 한국어 단어로 번역

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
tf.reset_default_graph()
#S:디코딩 입력의 시작, E:디코딩 출력의 끝
#P:패딩, time step크기보다 작은 단어의 경우, 빈 자리를 
#채우는 문자
#time step=4인 경우, love -> l, o, v, e
# my -> m, y, P, P
char_arr=[c for c in "SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀목록사랑"]
#char_arr #코퍼스
num_dic={n:i for i,n in enumerate(char_arr)} #{1:'S', ...}
#num_dic
dic_len=len(num_dic) #41

In [4]:
#영어 단어를 한글로 번역하기 위한 학습용 데이터
seq_data=[['word', '단어'], ['tree', '나무'],
         ['game', '놀이'],['girl', '소녀'],
         ['list', '목록'],['love', '사랑']]

In [5]:
def make_batch(seq_data): # ('word','PPPP')
    input_batch=[]
    output_batch=[]
    target_batch=[]
    for seq in seq_data:
        #인코더 셀의 입력값
        input=[num_dic[n] for n in seq[0]]
        #디코더 셀의 입력값, S:시작
        output=[num_dic[n] for n in ('S'+seq[1])]
        #디코더 셀의 출력값, E:끝
        target=[num_dic[n] for n in (seq[1]+'E')]
        #print(target)
        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)
        print(target_batch)    
        #출력값만 원핫 인코딩이 아님.
        #sparse_softmax_cross_entropy_with_logits사용
    return input_batch, output_batch, target_batch

learning_rate=0.01
n_hidden=128
total_epoch=100
#입력과 출력의 형태는 one-hot 인코딩으로 크기가 같음.
n_class=n_input=dic_len #41

#신경망 구성
#seq2seq모델은 인코더의 입력과 디코더의 형식이 같음
#[batch size, time step, input size]
#디코더의 출력형식
#[batch size, time step]
enc_input=tf.placeholder(tf.float32, [None, None, n_input])
#배치크기, 글자 길이가 정해지 있지 않음
#같은 배치인 경우에는 입력 데이터의 글자는 모두 같아야 함
dec_input=tf.placeholder(tf.float32, [None, None, n_input])
targets=tf.placeholder(tf.int64, [None, None])

with tf.variable_scope('encode'):
    #인코더 셀 구성
    enc_cell=tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    #오버피팅 방지하기 위해 드랍아웃
    enc_cell=tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    #드라이버 설정
    outputs, enc_states=tf.nn.dynamic_rnn(enc_cell,enc_input,dtype=tf.float32)

with tf.variable_scope('decode'):
    #디코더 셀 구성
    dec_cell=tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell=tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    #드라이버 설정
    outputs, dec_states=tf.nn.dynamic_rnn(dec_cell,dec_input,initial_state=enc_states,dtype=tf.float32)
    #seq2seq 모델은 인코더 셀의 최종 상태 값이 디코더 셀의
    #초기 상태값으로 입력해줘야 함.initial_state=enc_states 

model=tf.layers.dense(outputs, n_class, activation=None)
#model수행 결과가 [batch size, time step, input]
cost=tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=targets))
optimizer=tf.train.AdamOptimizer(learning_rate).minimize(cost)
#디코더 셀 구성

#학습

sess=tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, output_batch, target_batch=make_batch(seq_data)

for epoch in range(total_epoch):
    _,loss=sess.run([optimizer, cost], feed_dict={
        enc_input:input_batch,
        dec_input:output_batch,
        targets:target_batch
    })
    print("epoch:", "%04d" % (epoch+1),
         "cost:", "{:.6f}".format(loss))
print("모델 완료")

#단어를 입력받아서 번역된 단어를 예측하고 디코딩하는 함수
def translate(word):
    #단어 번역    
    #[영어단어, 한글단어]
    
    #예측시에는 한글단어를 모르므로, 디코더의 입출력값을
    #의미없는 값인 P로 채운다
    #['word', 'PPPP']
    seq_data=[word, 'P'*len(word)]
    input_batch, output_batch, target_batch=make_batch([seq_data])
    #[batch size, time step, input]
    
    prediction=tf.argmax(model,2)
#model수행 결과가 [batch size, time step, input]
#2번째 차원인 input차원을 argmax로 해서 가장 확률이
#높은 글자를 예측값으로 함
    result=sess.run(prediction,feed_dict={
        enc_input:input_batch,
        dec_input:output_batch,
        targets:target_batch
    })
    decoded=[char_arr[i] for i in result[0]]
    #결과값이 숫자로 나옴. 숫자의 인덱스에 해당하는 문자 가져옴
    #출력의 끝 'E'이후의 글자들 제거하고 문자열 출력
    end=decoded.index('E')
    translated="".join(decoded[:end])
    
    return translated

print('word ->', translate('word')) #단어
print('wodr ->', translate('wodr')) #단어
print('love ->', translate('love')) #사랑
print('loev ->', translate('loev')) #사랑
print('abcd ->', translate('abcd')) #????


W0906 11:55:21.419607 15320 deprecation.py:323] From <ipython-input-5-dd0a2911d120>:40: BasicRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
W0906 11:55:21.425016 15320 deprecation.py:323] From <ipython-input-5-dd0a2911d120>:44: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0906 11:55:21.475886 15320 deprecation.py:506] From C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to 

[[29, 30, 1]]
[[29, 30, 1], [31, 32, 1]]
[[29, 30, 1], [31, 32, 1], [33, 34, 1]]
[[29, 30, 1], [31, 32, 1], [33, 34, 1], [35, 36, 1]]
[[29, 30, 1], [31, 32, 1], [33, 34, 1], [35, 36, 1], [37, 38, 1]]
[[29, 30, 1], [31, 32, 1], [33, 34, 1], [35, 36, 1], [37, 38, 1], [39, 40, 1]]
epoch: 0001 cost: 3.714258
epoch: 0002 cost: 2.554141
epoch: 0003 cost: 1.644774
epoch: 0004 cost: 1.046160
epoch: 0005 cost: 0.582317
epoch: 0006 cost: 0.338668
epoch: 0007 cost: 0.130535
epoch: 0008 cost: 0.073502
epoch: 0009 cost: 0.065292
epoch: 0010 cost: 0.039924
epoch: 0011 cost: 0.034746
epoch: 0012 cost: 0.017511
epoch: 0013 cost: 0.017655
epoch: 0014 cost: 0.012772
epoch: 0015 cost: 0.007895
epoch: 0016 cost: 0.009022
epoch: 0017 cost: 0.006562
epoch: 0018 cost: 0.007555
epoch: 0019 cost: 0.005660
epoch: 0020 cost: 0.002438
epoch: 0021 cost: 0.003515
epoch: 0022 cost: 0.001669
epoch: 0023 cost: 0.004012
epoch: 0024 cost: 0.001204
epoch: 0025 cost: 0.001888
epoch: 0026 cost: 0.001156
epoch: 0027 cost: 0